In [3]:
!pip uninstall keras-mxnet -y

In [2]:
!pip install keras

In [4]:
import json
import os
from pprint import pprint

with open(os.path.expanduser('~')+'/.keras/keras.json','rb') as f:
    d = json.load(f)
    print(d)

{'epsilon': 1e-07, 'image_data_format': 'channels_first', 'backend': 'mxnet', 'floatx': 'float32'}


In [5]:
import os

with open(os.path.expanduser('~')+'/.keras/keras.json','w') as f:
    new_settings = """{\r\n
    "epsilon": 1e-07,\r\n
    "image_data_format": "channels_last",\n
    "backend": "tensorflow",\r\n
    "floatx": "float32"\r\n
    }"""
    f.write(new_settings)

In [6]:
import json
import os
from pprint import pprint

with open(os.path.expanduser('~')+'/.keras/keras.json','rb') as f:
    d = json.load(f)
    print(d)

{'epsilon': 1e-07, 'image_data_format': 'channels_last', 'backend': 'tensorflow', 'floatx': 'float32'}


In [7]:
import os
import cv2
import numpy as np

In [8]:
# Define paths
base_dir = os.path.abspath('') # don't use dunder variable __file__ here. 

In [9]:
print(base_dir)

/Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens


In [4]:
prototxt_path = os.path.join(base_dir + '/model_data/deploy.prototxt')
caffemodel_path = os.path.join(base_dir + '/model_data/weights.caffemodel')
# Read the model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

In [5]:
# Create directory 'UpdatedImages' if it does not exist
if not os.path.exists(base_dir + '/UpdatedImages'):
    print("New directory created")
    os.makedirs(base_dir + '/UpdatedImages')

In [6]:
# Create directory 'Faces' if it does not exist
if not os.path.exists(base_dir + '/Faces'):
    print("New directory created")
    os.makedirs(base_dir + '/Faces')

In [9]:
# Loop through all images and save images with marked faces

for file in os.listdir(base_dir + '/Images'):
    file_name, file_extension = os.path.splitext(file)
    if (file_extension in ['.png','.jpg']):
        print("Image path: {}".format(base_dir + '/Images/' + file))
        image = cv2.imread(base_dir + '/Images/' + file)
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        model.setInput(blob)
        detections = model.forward()
        # Create frame around face
        for i in range(0, detections.shape[2]):
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            confidence = detections[0, 0, i, 2]
            # If confidence > 0.5, show box around face
            if (confidence > 0.5):
                cv2.rectangle(image, (startX, startY), (endX, endY), (255, 255, 255), 2)
                cv2.imwrite(base_dir + '/UpdatedImages/' + file, image)
                print("Image " + file + " converted successfully")
                count += 1
                frame = image[startY:endY, startX:endX]
                cv2.imwrite(base_dir + '/Faces/' + 'Face_' + file, frame)

Image path: /Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens/Images/vikram-24.jpg
Image vikram-24.jpg converted successfully
Image vikram-24.jpg converted successfully
Image path: /Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens/Images/ntr.jpg
Image ntr.jpg converted successfully
Image path: /Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens/Images/vik.jpg
Image vik.jpg converted successfully
Image vik.jpg converted successfully
Image path: /Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens/Images/Vikram_0-770x433.jpg
Image Vikram_0-770x433.jpg converted successfully
Image path: /Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens/Images/kar5.jpg
Image kar5.jpg converted successfully
Image path: /Users/Bodacious/PycharmProjects/MSDS_DeepLearning/FaceClassification_AWSDeepLens/Images/vikr.jpg
Image vikr.jpg converted successfully
Im

## Model for classification - facial images 

In [10]:
# Importing libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [11]:
train = pd.read_csv('TrainClassLabels.csv')

In [12]:
# We have color images, so while loading the images we will keep grayscale=false, 
#if you have greyscale images, you should set grayscale as true
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(base_dir + '/Faces/'+train['ImageName'][i], target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
X

100%|██████████| 146/146 [00:00<00:00, 1401.75it/s]


array([[[[0.9882353 , 0.98039216, 0.9843137 ],
         [1.        , 0.99607843, 1.        ],
         [1.        , 0.99607843, 1.        ],
         ...,
         [0.99215686, 0.99607843, 1.        ],
         [1.        , 0.99607843, 1.        ],
         [0.99607843, 0.99607843, 1.        ]],

        [[1.        , 1.        , 0.99215686],
         [0.654902  , 0.6509804 , 0.6431373 ],
         [0.6509804 , 0.64705884, 0.6392157 ],
         ...,
         [0.16078432, 0.15686275, 0.18039216],
         [0.09411765, 0.09019608, 0.11372549],
         [0.13725491, 0.13333334, 0.15686275]],

        [[1.        , 0.99607843, 0.9882353 ],
         [0.61960787, 0.6156863 , 0.6       ],
         [0.64705884, 0.627451  , 0.6117647 ],
         ...,
         [0.09803922, 0.09411765, 0.11764706],
         [0.12941177, 0.1254902 , 0.15686275],
         [0.11764706, 0.11372549, 0.13725491]],

        ...,

        [[1.        , 0.99607843, 1.        ],
         [0.8509804 , 0.84313726, 0.8627451 ]

In [13]:
print('X shape:', X.shape)

X shape: (146, 28, 28, 3)


In [14]:
train

,ImageName,PersonName
0,ntr1.jpg,NTR
1,ntr2.jpg,NTR
2,ntr3.jpg,NTR
3,ntr4.jpg,NTR
4,ntr5.jpg,NTR
5,ntr6.jpg,NTR
6,ntr7.jpg,NTR
7,ntr8.jpg,NTR
8,ntr9.jpg,NTR
9,ntr10.jpg,NTR


In [15]:
from keras.utils import to_categorical

In [16]:
num_classes = 3
train['PersonName'] = train['PersonName'].astype('category')
train['PersonName_catcode'] = train['PersonName'].cat.codes
y = train['PersonName_catcode'].values
y = keras.utils.to_categorical(y, num_classes)
y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1

In [17]:
train

,ImageName,PersonName,PersonName_catcode
0,ntr1.jpg,NTR,1
1,ntr2.jpg,NTR,1
2,ntr3.jpg,NTR,1
3,ntr4.jpg,NTR,1
4,ntr5.jpg,NTR,1
5,ntr6.jpg,NTR,1
6,ntr7.jpg,NTR,1
7,ntr8.jpg,NTR,1
8,ntr9.jpg,NTR,1
9,ntr10.jpg,NTR,1


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [20]:
from keras.layers import Activation, BatchNormalization, LeakyReLU

model = Sequential()

ActRelu = Activation('relu')
ActSoftmax = Activation('softmax')

FirstConv2DLayer = Conv2D(32, (3,3), input_shape=(28,28,3))
model.add(FirstConv2DLayer)
BatchNormalization(axis=-1) #Axis -1 is always features axis
model.add(ActRelu)

model.add(Conv2D(64, (3,3)))
model.add(ActRelu)

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

model.add(Flatten())
BatchNormalization()
model.add(Dense(512))
BatchNormalization()
model.add(ActRelu)

model.add(Dropout(0.2))
model.add(Dense(3))
model.add(ActSoftmax)

In [21]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [22]:
from keras.preprocessing.image import ImageDataGenerator
# applying transformation to image
train_gen = ImageDataGenerator(rotation_range=8, 
                               width_shift_range=0.08, 
                               shear_range=0.3, 
                               height_shift_range=0.08, 
                               zoom_range=0.08 )
test_gen = ImageDataGenerator()

In [23]:
training_set= train_gen.flow(X_train, y_train, batch_size=10)
test_set= train_gen.flow(X_test, y_test, batch_size=10)

In [24]:
model.fit_generator(training_set, steps_per_epoch=60000//64, 
                              validation_data= test_set, 
                              validation_steps=10000//64, 
                              epochs=25
                             )


Epoch 1/25
937/937 [==============================] - 42s 45ms/step - loss: 0.1769 - accuracy: 0.9181 - val_loss: 0.7272 - val_accuracy: 0.7987
Epoch 2/25
937/937 [==============================] - 44s 47ms/step - loss: 0.0132 - accuracy: 0.9955 - val_loss: 1.7245 - val_accuracy: 0.7647
Epoch 3/25
937/937 [==============================] - 46s 49ms/step - loss: 0.0051 - accuracy: 0.9981 - val_loss: 2.3833 - val_accuracy: 0.8205
Epoch 4/25
937/937 [==============================] - 46s 49ms/step - loss: 0.0033 - accuracy: 0.9992 - val_loss: 2.1461 - val_accuracy: 0.8032
Epoch 5/25
937/937 [==============================] - 47s 50ms/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 1.3304 - val_accuracy: 0.8006
Epoch 6/25
937/937 [==============================] - 46s 49ms/step - loss: 0.0015 - accuracy: 0.9990 - val_loss: 2.1553 - val_accuracy: 0.8224
Epoch 7/25
937/937 [==============================] - 47s 50ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 1.4306 - val_accuracy

In [25]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 80.00%


In [26]:
test2 = pd.read_csv('TestClassLabels.csv')

In [27]:
test2

,ImageName,PersonName
0,ntr61.jpg,NaN
1,ntr62.jpg,NaN
2,ntr63.jpg,NaN
3,ntr64.jpg,NaN
4,ntr65.jpg,NaN
5,ntr66.jpg,NaN
6,ntr67.jpg,NaN
7,ntr68.jpg,NaN
8,ntr69.jpg,NaN
9,ntr70.jpg,NaN


In [28]:
test_image = []
for i in tqdm(range(test2.shape[0])):
    img = image.load_img('TestFaces/'+test2['ImageName'][i], target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test2 = np.array(test_image)

100%|██████████| 38/38 [00:00<00:00, 132.11it/s]


In [29]:
# making predictions
prediction = model.predict_classes(test2)

In [30]:
prediction

array([2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2,
       2, 2, 1, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2])

In [34]:
model.save('faceclassification_tf.pb')

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
activation_3 (Activation)    multiple                  0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

In [36]:
import boto3
import re

assumed_role = boto3.client('sts').get_caller_identity()['Arn']
s3_access_role = re.sub(r'^(.+)sts::(\d+):assumed-role/(.+?)/.*$', r'\1iam::\2:role/\3', assumed_role)
print(s3_access_role)
s3 = boto3.resource('s3')
bucket= 'deeplens-sagemaker-faceclassification/facerecognition_tf' 

pb = open(base_dir+'/faceclassification.pb', 'rb')
s3.Bucket(bucket).put_object(Key='faceclassification.pb', Body=pb)

NoCredentialsError: Unable to locate credentials